<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Глубокое обучение. Часть 2
# Домашнее задание по теме "Механизм внимания"

Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы.

В этом задании вы будете решать задачу классификации математических задач по темам (многоклассовая классификация) с помощью Transformer.

В качестве датасета возьмем датасет математических задач по разным темам. Нам необходим следующий файл:

[Файл с классами](https://docs.google.com/spreadsheets/d/13YIbphbWc62sfa-bCh8MLQWKizaXbQK9/edit?usp=drive_link&ouid=104379615679964018037&rtpof=true&sd=true)

**Hint:** не перезаписывайте модели, которые вы получите на каждом из этапов этого дз. Они ещё понадобятся.

### Задание 1 (2 балла)

Напишите кастомный класс для модели трансформера для задачи классификации, использующей в качествке backbone какую-то из моделей huggingface.

Т.е. конструктор класса должен принимать на вход название модели и подгружать её из huggingface, а затем использовать в качестве backbone (достаточно возможности использовать в качестве backbone те модели, которые упомянуты в последующих пунктах)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.ticker as ticker
from transformers import get_scheduler
import matplotlib.pyplot as plt

import numpy as np

import random
import math
import time
import string
import pymorphy2
import re

# datasets from huggingface
import transformers
from datasets import load_dataset
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding

from nltk.corpus import stopwords
import nltk

from tqdm.notebook import tqdm
# from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

import os
from datetime import datetime
from torch.nn.utils.rnn import pad_sequence

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [18]:
!pip install accelerate -U -q

In [19]:
!pip install datasets -q

In [20]:
!pip install transformers -q

In [21]:
!pip install evaluate -q

In [2]:
class TransformerClassificationModel(nn.Module):
    #Будем передавать в kwargs еще и num_layers, тогда дополнительный полносвязный слой не понадобится
    def __init__(self, base_transformer_model_huggingface_name=None, base_transformer_model=None, **kwargs):
        assert base_transformer_model is not None or base_transformer_model_huggingface_name is not None
        super().__init__()
        if base_transformer_model_huggingface_name is not None:
            # kwargs['num_labels'] = 1
            self.backbone = transformers.AutoModelForSequenceClassification.from_pretrained(base_transformer_model_huggingface_name, kwargs)
        else:
            self.backbone = base_transformer_model
        
    def forward(self, inputs):
        # YOUR CODE: propagate inputs through the model. Return dict with logits

        outputs = self.backbone(inputs)
        return outputs

### Задание 2 (1 балл)

Напишите функцию заморозки backbone у модели (если необходимо, возвращайте из функции модель)

In [3]:
def freeze_backbone_function(model: TransformerClassificationModel):
    for w in model.backbone.parameters():
        w._trainable = False
    for w in model.classifier.parameters():
        w._trainable = True

### Задание 3 (2 балла)

Напишите функцию, которая будет использована для тренировки (дообучения) трансформера (TransformerClassificationModel). Функция должна поддерживать обучение с замороженным и размороженным backbone.

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [5]:
dataset = load_dataset("csv", data_files="C:\\Users\\koman\\jupiter_prj\\Deep-Learning-School\\NLP\\my_projects\\hw4-6 transformers_and_finetuning\\data_problems_translated.csv")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'problem_text', 'topic'],
        num_rows: 5273
    })
})

In [6]:
dataset = dataset.remove_columns('Unnamed: 0')

In [7]:
label_vocab = set()
for w in dataset['train']['topic']:
    label_vocab.add(w)

label2id = {char: i for i, char in enumerate(label_vocab)}
id2label = {i: char for i, char in enumerate(label_vocab)}

label2id

{'number_theory': 0,
 'graphs': 1,
 'polynoms': 2,
 'dirichlet': 3,
 'geometry': 4,
 'combinatorics': 5,
 'invariant': 6}

In [8]:
dataset = dataset['train'].train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['problem_text', 'topic'],
        num_rows: 4745
    })
    test: Dataset({
        features: ['problem_text', 'topic'],
        num_rows: 528
    })
})

In [9]:
def preprocess_function(examples):
    examples['labels'] = [label2id[example] for example in examples['topic']]
    return tokenizer(examples["problem_text"], truncation=True)

In [10]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4745 [00:00<?, ? examples/s]

Map:   0%|          | 0/528 [00:00<?, ? examples/s]

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=id2label.__len__(), id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

C:\Users\koman\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\koman\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at dis

In [29]:
from transformers import TrainingArguments, Trainer
import evaluate
import accelerate

In [31]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [32]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [30]:
transformers.__version__, accelerate.__version__

('4.41.2', '0.27.2')

In [ ]:
import copy

def train_transformer(transformer_model, freeze_backbone=True)
    model = copy.copy(transformer_model)
    ### YOUR CODE IS HERE

    return finetuned_model

### Задание 4 (1 балл)

Проверьте вашу функцию из предыдущего пункта, дообучив двумя способами
*cointegrated/rubert-tiny2* из huggingface.

In [ ]:
rubert_tiny_transformer_model = #...
rubert_tiny_finetuned_with_freezed_backbone = train_transformer(rubert_tiny_transformer_model, freeze_backbone=True)

rubert_tiny_transformer_model = #...
rubert_tiny_full_finetuned = train_transformer(rubert_tiny_transformer_model, freeze_backbone=False)

### Задание 5 (1 балл)

Обучите *tbs17/MathBert* (с замороженным backbone и без заморозки), проанализируйте результаты. Сравните скоры с первым заданием. Получилось лучше или нет? Почему?

In [ ]:
### YOUR CODE IS HERE (probably, similar on the previous step)

### Задание 6 (1 балл)

Напишите функцию для отрисовки карт внимания первого слоя для моделей из задания

In [ ]:
def draw_first_layer_attention_maps(attention_head_ids: List, text: str, model: TransformerClassificationModel):
    pass

### Задание 7 (1 балл)

Проведите инференс для всех моделей **ДО ДООБУЧЕНИЯ** на 2-3 текстах из датасета. Посмотрите на головы Attention первого слоя в каждой модели на выбранных текстах (отрисуйте их отдельно).

Попробуйте их проинтерпретировать. Какие связи улавливают карты внимания? (если в модели много голов Attention, то проинтерпретируйте наиболее интересные)

In [ ]:
### YOUR CODE IS HERE

### Задание 8 (1 балл)

Сделайте то же самое для дообученных моделей. Изменились ли карты внимания и связи, которые они улавливают? Почему?

In [ ]:
### YOUR CODE IS HERE